In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


/data/user/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/pyiomica_master/pyiomica/visibilityGraphAuxiliaryFunctions.py:91: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(cache=True)


In [2]:
x_frame1_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame1_1.25_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame2_1.25_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame3_1.25_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/sahaj432/Desktop/dataset/neural_4_class/x_frame_noise_1.25_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05[:6000]),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05[:6000]))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "natural", weight='tan')
        vs_out=np.asarray(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

533


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
print(len(x_adj))

25578


In [11]:
x_adj=np.array(x_adj)

In [12]:
def degree(data): #The node strengths (or weighted degrees) 
    return np.average(np.sum(data, axis=0))

In [13]:
import networkx as nx
def runall(data):
    temp1=[]
    for i in data:
        G = nx.from_numpy_array(i, parallel_edges=False)
        
        t=[nx.transitivity(G)]
        ge=[nx.global_efficiency(G)]
        le=[nx.local_efficiency(G)]
        
        cc=list(nx.clustering(G).values())
        temp1.append(t+ge+le+cc) #+cpl+ge+le+ac+cc
    return temp1

In [14]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
533


In [15]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(runall, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 19.922739028930664 seconds ---


In [16]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [17]:
x_g=np.array(x_g)


In [18]:
print(x_g.shape)

(25578, 59)


In [19]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [20]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(17904, 59)
(3837, 59)
(3837, 59)


In [21]:
import tensorflow as tf


2023-08-13 09:26:26.266768: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:26:26.266807: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [22]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 09:26:37.355172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 09:26:37.355246: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 09:26:37.355281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0191): /proc/driver/nvidia/version does not exist
2023-08-13 09:26:37.355809: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 59, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 29, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 448)               0         
                                                                 
 dense (Dense)               (None, 100)               4

In [24]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F5.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [26]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
560/560 [==============================] - 2s 3ms/step - loss: 0.4619 - acc: 0.8084 - val_loss: 0.2370 - val_acc: 0.9072
Epoch 2/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2428 - acc: 0.9026 - val_loss: 0.2268 - val_acc: 0.9132
Epoch 3/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2227 - acc: 0.9110 - val_loss: 0.2100 - val_acc: 0.9130
Epoch 4/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2111 - acc: 0.9164 - val_loss: 0.2670 - val_acc: 0.8999
Epoch 5/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2061 - acc: 0.9197 - val_loss: 0.2248 - val_acc: 0.9049
Epoch 6/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2014 - acc: 0.9205 - val_loss: 0.1984 - val_acc: 0.9208
Epoch 7/30
560/560 [==============================] - 2s 3ms/step - loss: 0.1928 - acc: 0.9233 - val_loss: 0.2087 - val_acc: 0.9195
Epoch 8/30
560/560 [==============================] - 2s 3ms/step - loss: 0.

Epoch 29/30
560/560 [==============================] - 2s 3ms/step - loss: 0.0844 - acc: 0.9670 - val_loss: 0.2872 - val_acc: 0.9096
Epoch 30/30
560/560 [==============================] - 2s 3ms/step - loss: 0.0855 - acc: 0.9679 - val_loss: 0.2910 - val_acc: 0.9093


In [27]:
model3.save_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/F5.h5')


In [28]:
model3.load_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_neural_code_SNR_1.25_v/F5.h5')


In [29]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [30]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[950   2   0  11]
 [  0 831 157  13]
 [  0  80 902   0]
 [ 59  26   0 806]]


In [31]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [32]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

90.93041438623925
91.01837638272262
90.93041438623925
91.06620881792955
